In [1]:
import tkinter as tk
import threading
import time
from pythonosc import udp_client
from scipy.spatial.transform import Rotation as R
import openvr
import keyboard

measuring_height = 1.13

def main_loop(trackerID, hostAdress, ports, Hz, ear_height_offset, update_text_area):
    # OSC Clients
    roomEncoderOSC = udp_client.SimpleUDPClient(hostAdress, ports['roomEncoder_port'])
    roomEncoderOSC2 = udp_client.SimpleUDPClient(hostAdress, ports['roomEncoder_port2'])
    roomEncoderOSC3 = udp_client.SimpleUDPClient(hostAdress, ports['roomEncoder_port3'])

    sceneRotatorOSC = udp_client.SimpleUDPClient(hostAdress, ports['sceneRotator_port'])
    sceneRotatorOSC2 = udp_client.SimpleUDPClient(hostAdress, ports['sceneRotator_port2'])
    sceneRotatorOSC3 = udp_client.SimpleUDPClient(hostAdress, ports['sceneRotator_port3'])
    sceneRotatorOSC4 = udp_client.SimpleUDPClient(hostAdress, ports['sceneRotator_port4'])
    sceneRotatorOSC5 = udp_client.SimpleUDPClient(hostAdress, ports['sceneRotator_port5'])
    sceneRotatorOSC6 = udp_client.SimpleUDPClient(hostAdress, ports['sceneRotator_port6'])

    # Initialize OpenVR
    openvr.init(openvr.VRApplication_Other)

    def get_tracker_pose(tracker_index):
        poses = openvr.VRSystem().getDeviceToAbsoluteTrackingPose(
            openvr.TrackingUniverseStanding, 0, openvr.k_unMaxTrackedDeviceCount)
        
        if poses[tracker_index].bPoseIsValid:
            pose = poses[tracker_index].mDeviceToAbsoluteTracking
            return pose
        else:
            return None

    def extract_position_and_rotation(pose):
        posY = pose[0][3]  # Vive X = IEM Y
        posZ = pose[1][3]  # Vive Y = IEM Z
        posX = pose[2][3]  # Vive Z = IEM X

        r = R.from_matrix([[pose[2][0], pose[2][1], pose[2][2]],
                           [pose[0][0], pose[0][1], pose[0][2]],
                           [pose[1][0], pose[1][1], pose[1][2]]])

        euler = r.as_euler('zyx', degrees=True)
        
        return (posX, posY, posZ), euler

    offset_position = (0, 0, 0)
    offset_ypr = (0, 0, 0)

    # Main loop
    Loop = True
    try:
        while Loop:
            if keyboard.is_pressed('r'):
                update_text_area("Position und Rotation zurückgesetzt.")
                offset_position, offset_ypr = extract_position_and_rotation(get_tracker_pose(trackerID))
            
            if keyboard.is_pressed('q'):
                Loop = False
                update_text_area("Programm beendet.")

            pose = get_tracker_pose(trackerID)
            if pose:
                position, ypr = extract_position_and_rotation(pose)
                
                relative_position = (position[0] - offset_position[0], 
                                     position[1] - offset_position[1], 
                                     position[2] - offset_position[2]+ear_height_offset)
                
                relative_ypr = (ypr[0] - offset_ypr[0], 
                                ypr[1] - offset_ypr[1], 
                                ypr[2] - offset_ypr[2])

                # For debugging
                # update_text_area(f"Position X: {relative_position[0]:.2f}, Y: {relative_position[1]:.2f}, Z: {relative_position[2]:.2f}")

                roomEncoderOSC.send_message(f"/RoomEncoder/listenerX", relative_position[0])
                roomEncoderOSC.send_message(f"/RoomEncoder/listenerY", relative_position[1])
                roomEncoderOSC.send_message(f"/RoomEncoder/listenerZ", relative_position[2])

                roomEncoderOSC2.send_message(f"/RoomEncoder/listenerX", relative_position[0])
                roomEncoderOSC2.send_message(f"/RoomEncoder/listenerY", relative_position[1])
                roomEncoderOSC2.send_message(f"/RoomEncoder/listenerZ", relative_position[2])

                roomEncoderOSC3.send_message(f"/RoomEncoder/listenerX", relative_position[0])
                roomEncoderOSC3.send_message(f"/RoomEncoder/listenerY", relative_position[1])
                roomEncoderOSC3.send_message(f"/RoomEncoder/listenerZ", relative_position[2])
                    
                sceneRotatorOSC.send_message(f"/SceneRotator/yaw", relative_ypr[0])
                sceneRotatorOSC.send_message(f"/SceneRotator/pitch", relative_ypr[1])

                sceneRotatorOSC2.send_message(f"/SceneRotator/yaw", relative_ypr[0])
                sceneRotatorOSC2.send_message(f"/SceneRotator/pitch", relative_ypr[1])

                sceneRotatorOSC3.send_message(f"/SceneRotator/yaw", relative_ypr[0])
                sceneRotatorOSC3.send_message(f"/SceneRotator/pitch", relative_ypr[1])

                sceneRotatorOSC4.send_message(f"/SceneRotator/yaw", relative_ypr[0])
                sceneRotatorOSC4.send_message(f"/SceneRotator/pitch", relative_ypr[1])

                sceneRotatorOSC5.send_message(f"/SceneRotator/yaw", relative_ypr[0])
                sceneRotatorOSC5.send_message(f"/SceneRotator/pitch", relative_ypr[1])

                sceneRotatorOSC6.send_message(f"/SceneRotator/yaw", relative_ypr[0])
                sceneRotatorOSC6.send_message(f"/SceneRotator/pitch", relative_ypr[1])

                time.sleep(1/Hz)

    except KeyboardInterrupt:
        update_text_area("Programm unterbrochen.")

    finally:
        openvr.shutdown()
        update_text_area("Beendet.")





# GUI
def create_gui():
    root = tk.Tk()
    root.title("Tracking2OSC")

    # Input text boxes
    tk.Label(root, text="Tracker ID:").grid(row=0, column=0, sticky=tk.W)
    tracker_id_entry = tk.Entry(root)
    tracker_id_entry.insert(0, "1")
    tracker_id_entry.grid(row=0, column=1)

    tk.Label(root, text="Host Address:").grid(row=1, column=0, sticky=tk.W)
    host_entry = tk.Entry(root)
    host_entry.insert(0, "127.0.0.1")
    host_entry.grid(row=1, column=1)

    tk.Label(root, text="Room Encoder Port 1:").grid(row=2, column=0, sticky=tk.W)
    roomEncoder_port_entry = tk.Entry(root)
    roomEncoder_port_entry.insert(0, "7000")
    roomEncoder_port_entry.grid(row=2, column=1)

    tk.Label(root, text="Room Encoder Port 2:").grid(row=3, column=0, sticky=tk.W)
    roomEncoder_port2_entry = tk.Entry(root)
    roomEncoder_port2_entry.insert(0, "7001")
    roomEncoder_port2_entry.grid(row=3, column=1)

    tk.Label(root, text="Room Encoder Port 3:").grid(row=4, column=0, sticky=tk.W)
    roomEncoder_port3_entry = tk.Entry(root)
    roomEncoder_port3_entry.insert(0, "7002")
    roomEncoder_port3_entry.grid(row=4, column=1)

    tk.Label(root, text="Scene Rotator Port 1:").grid(row=5, column=0, sticky=tk.W)
    sceneRotator_port_entry = tk.Entry(root)
    sceneRotator_port_entry.insert(0, "8000")
    sceneRotator_port_entry.grid(row=5, column=1)

    tk.Label(root, text="Scene Rotator Port 2:").grid(row=6, column=0, sticky=tk.W)
    sceneRotator_port2_entry = tk.Entry(root)
    sceneRotator_port2_entry.insert(0, "8001")
    sceneRotator_port2_entry.grid(row=6, column=1)

    tk.Label(root, text="Scene Rotator Port 3:").grid(row=7, column=0, sticky=tk.W)
    sceneRotator_port3_entry = tk.Entry(root)
    sceneRotator_port3_entry.insert(0, "8002")
    sceneRotator_port3_entry.grid(row=7, column=1)

    tk.Label(root, text="Scene Rotator Port 4:").grid(row=8, column=0, sticky=tk.W)
    sceneRotator_port4_entry = tk.Entry(root)
    sceneRotator_port4_entry.insert(0, "8003")
    sceneRotator_port4_entry.grid(row=8, column=1)

    tk.Label(root, text="Scene Rotator Port 5:").grid(row=9, column=0, sticky=tk.W)
    sceneRotator_port5_entry = tk.Entry(root)
    sceneRotator_port5_entry.insert(0, "8004")
    sceneRotator_port5_entry.grid(row=9, column=1)

    tk.Label(root, text="Scene Rotator Port 6:").grid(row=10, column=0, sticky=tk.W)
    sceneRotator_port6_entry = tk.Entry(root)
    sceneRotator_port6_entry.insert(0, "8005")
    sceneRotator_port6_entry.grid(row=10, column=1)

    tk.Label(root, text="Hz:").grid(row=11, column=0, sticky=tk.W)
    Hz_entry = tk.Entry(root)
    Hz_entry.insert(0, "90")
    Hz_entry.grid(row=11, column=1)

    tk.Label(root, text="Ear Height (m):").grid(row=12, column=0, sticky=tk.W)
    ear_height_entry = tk.Entry(root)
    ear_height_entry.insert(0, "1.13")
    ear_height_entry.grid(row=12, column=1)

    # Text output
    output_text = tk.Text(root, height=15, width=50)
    output_text.grid(row=13, column=0, columnspan=2, pady=10)

    def update_text_area(message):
        output_text.insert(tk.END, message + "\n")
        output_text.see(tk.END)

    def start_program():
        trackerID = int(tracker_id_entry.get())
        hostAdress = host_entry.get()
        ports = {
            'roomEncoder_port': int(roomEncoder_port_entry.get()),
            'roomEncoder_port2': int(roomEncoder_port2_entry.get()),
            'roomEncoder_port3': int(roomEncoder_port3_entry.get()),
            'sceneRotator_port': int(sceneRotator_port_entry.get()),
            'sceneRotator_port2': int(sceneRotator_port2_entry.get()),
            'sceneRotator_port3': int(sceneRotator_port3_entry.get()),
            'sceneRotator_port4': int(sceneRotator_port4_entry.get()),
            'sceneRotator_port5': int(sceneRotator_port5_entry.get()),
            'sceneRotator_port6': int(sceneRotator_port6_entry.get()),
        }
        Hz = int(Hz_entry.get())
        ear_height_offset = float(ear_height_entry.get()) - measuring_height

        update_text_area("OSC Clients gestartet.")
        update_text_area("Drücke R zum Zurücksetzen der Orientierung. Drücke Q zum Beenden.")

        # Start main loop
        threading.Thread(target=main_loop, args=(trackerID, hostAdress, ports, Hz, ear_height_offset, update_text_area)).start()


    # Start button
    start_button = tk.Button(root, text="Start", command=start_program)
    start_button.grid(row=14, column=0, columnspan=2)

    root.mainloop()

# Start GUI
create_gui()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\oflan\miniconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\oflan\AppData\Local\Temp\ipykernel_9780\1369686605.py", line 220, in start_program
    ear_height_offset = float(ear_height_entry.get()) - measuring_height
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: '1.25r'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\oflan\miniconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\oflan\AppData\Local\Temp\ipykernel_9780\1369686605.py", line 220, in start_program
    ear_height_offset = float(ear_height_entry.get()) - measuring_height
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: '1.25r'
Exception in Tkinter callbac